# Standard Neural DAG Circuit Generator

This notebook explores a basic directed-acyclic graph (DAG) neural circuit generator, whose goal is to find circuits which accurately predict while also optimizing for minimal circuit geometry (edge and node count).

## TODO: Write more here to explain the experiment and its results

In [ ]:
import sys
from pathlib import Path

notebook_dir = Path.cwd()
project_root = notebook_dir.parent
sys.path.insert(0, str(project_root))

print(f"Project root added to sys.path: {str(project_root)}")
print(f"Current sys.path: {sys.path}")

import numpy
import cktgen
import random
import typing
import itertools
import dataclasses
import matplotlib.pyplot as plt

In [ ]:
def literal_combinations(literal_count: int) -> list[list[float]]:
    return [list(map(float, p)) for p in itertools.product([0, 1], repeat=literal_count)]

def createNeuralDAG(
        input_data: list[list[float]],
        output_data: list[list[float]],
        max_hidden_node_count: int = 5,
        epochs: int = 100000,
        learning_rate: float = 0.05,
        lambda_edges: float = 1e-2,
        lambda_nodes: float = 1e-2,
        node_strength_sharpness: float = 10.0,
    ) -> cktgen.models.StandardNeuralDAG:
    assert len(input_data) == len(output_data), "Malformed input-output pairs!"

    dag_model: cktgen.models.StandardNeuralDAG = cktgen.models.StandardNeuralDAG(
        cktgen.models.StandardNeuralDAGCreateInfo(
            input_node_count=len(input_data[0]) if len(input_data) > 0 else 0,
            hidden_node_count=max_hidden_node_count,
            output_node_count=len(output_data[0]) if len(output_data) > 0 else 0,
        )
    )

    dag_model.train(cktgen.models.StandardNeuralDAGTrainInfo(
        input_data=input_data,
        output_data=output_data,
        epochs=epochs,
        epoch_print_cadence=epochs + 1, # ? Effectivly Removes Printing
        learning_rate=learning_rate,
        lambda_edges=lambda_edges,
        lambda_nodes=lambda_nodes,
        node_strength_sharpness=node_strength_sharpness,
    ))

    return dag_model

def testAccuracyNeuralDAG(
    input_data: list[list[float]],
    output_data: list[list[float]],
    model: cktgen.models.StandardNeuralDAG,
    cleanOutput: typing.Callable[[float], float] = lambda val: float(round(val)),
    compare_epsilon: float = 1e-6,
) -> float:
    assert len(input_data) == len(output_data), "Malformed input-output pairs!"

    total_tests: float = float(len(input_data))
    if total_tests <= 0.0:
        return 1.0
    
    correct_results: float = 0.0
    for x, y in zip(input_data, output_data):
        output: list[float] = model.evaluate(x)

        for true_val, pred_val in zip(y, output):
            clean: float = cleanOutput(pred_val)

            if abs(true_val - clean) <= compare_epsilon:
                correct_results += 1.0

    return correct_results / total_tests

def visualizeNeuralDAG(
    filename: str,
    model: cktgen.models.StandardNeuralDAG,
    edge_prune_threshold: float = 1e-1,
) -> cktgen.graphs.NeuralGraph:
    dag: cktgen.graphs.NeuralGraph = model.extractDAG(edge_prune_threshold)
    dag.renderGraph(filename)
    return dag

In [ ]:
@dataclasses.dataclass
class TestDataNeuralDAG:
    test_name: str
    accuracy: float
    computation_node_count: int

__cur_test_data: list[TestDataNeuralDAG] = []

def pushTestNeuralDAG(test_name: str, input_data: list[list[float]], output_data: list[list[float]]) -> None:
    print(f"Testing {test_name} Neural DAG...")
    model: cktgen.models.StandardNeuralDAG = createNeuralDAG(input_data, output_data)
    dag: cktgen.graphs.NeuralGraph = visualizeNeuralDAG(f"{test_name}_dag", model)

    accuracy: float = testAccuracyNeuralDAG(input_data, output_data, model)
    
    hidden_node_count: int = dag.getHiddenNodeCount()
    output_node_count: int = dag.getOutputNodeCount()
    computation_node_count: int = hidden_node_count + output_node_count

    test_data: TestDataNeuralDAG = TestDataNeuralDAG(test_name, accuracy, computation_node_count)
    __cur_test_data.append(test_data)

def processTestData() -> None:
    sorted_data: list[TestDataNeuralDAG] = sorted(__cur_test_data, key=lambda datum: datum.computation_node_count)

    test_names: list[str] = [datum.test_name for datum in sorted_data]
    accuracies: list[float] = [datum.accuracy for datum in sorted_data]
    computation_node_counts: list[int] = [datum.computation_node_count for datum in sorted_data]

    __cur_test_data.clear()

    name_range = numpy.arange(len(test_names))
    name_width: float = 0.35

    plt.figure()
    plt.bar(name_range - (name_width * 0.5), computation_node_counts, name_width)
    plt.xticks(name_range, test_names, rotation=45, ha="right")
    plt.xlabel("Model Name")
    plt.ylabel("Computable Node Count")
    plt.title("Individual Model Complexity")
    plt.tight_layout()
    plt.show()

    plt.figure()
    plt.bar(name_range - (name_width * 0.5), accuracies, name_width)
    plt.xticks(name_range, test_names, rotation=45, ha="right")
    plt.xlabel("Model Name")
    plt.ylabel("Model Accuracy (%)")
    plt.title("Individual Model Accuracy")
    plt.tight_layout()
    plt.show()

    plt.figure()
    plt.scatter(computation_node_counts, accuracies)
    plt.xlabel("Computable Node Count")
    plt.ylabel("Model Accuracy (%)")
    plt.title("Model Complexity Vs Accuracy")
    plt.tight_layout()
    plt.show()

In [ ]:
two_literal_pairs: list[list[float]] = literal_combinations(2)

print("Testing Common 2-Input Boolean Functions\n------------------------------")
pushTestNeuralDAG("const_false", two_literal_pairs, [[0.0] for _ in two_literal_pairs])
pushTestNeuralDAG("const_true",  two_literal_pairs, [[1.0] for _ in two_literal_pairs])
pushTestNeuralDAG("and",         two_literal_pairs, [[0.0], [0.0], [0.0], [1.0]])
pushTestNeuralDAG("or",          two_literal_pairs, [[0.0], [1.0], [1.0], [1.0]])
pushTestNeuralDAG("nand",        two_literal_pairs, [[1.0], [1.0], [1.0], [0.0]])
pushTestNeuralDAG("nor",         two_literal_pairs, [[1.0], [0.0], [0.0], [0.0]])
pushTestNeuralDAG("xor",         two_literal_pairs, [[0.0], [1.0], [1.0], [0.0]])
processTestData()

In [ ]:
four_literal_pairs: list[list[float]] = literal_combinations(4)
four_random_test_count: int = 20

print("Testing Random 4-Input Boolean Functions\n------------------------------")
for test_idx in range(four_random_test_count):
    pushTestNeuralDAG(
        f"four_random_test_{test_idx}",
        four_literal_pairs,
        [[float(round(random.random()))] for _ in four_literal_pairs],
    )
processTestData()